In [1]:
type(spark)

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,Current session?
0,application_1603578626031_0002,pyspark,idle,Link,Link,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

<class 'pyspark.sql.session.SparkSession'>

In [2]:
phi_df = spark. \
    read. \
    format("csv"). \
    option("inferSchema","true"). \
    option("header", "true"). \
    load('s3://covid-19-tracker-2020/data/phi_CA.csv')

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [3]:
jh_df = spark. \
    read. \
    format("csv"). \
    schema("jh_Date date, jh_Country string, jh_Province string, jh_Lat double, jh_Long double, jh_Confirmed integer, \
            jh_Recovered integer, jh_Deaths integer"). \
    option("header", "true"). \
    load('s3://covid-19-tracker-2020/data/john_hopkins.csv')

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [4]:
phi_df1 = phi_df.select(phi_df.date.alias('phi_Date'), \
                        phi_df.prname.alias('phi_Province'), \
                        phi_df.numconf.alias('phi_Confirmed'), \
                        phi_df.numdeaths.alias('phi_Deaths') \
                       ). \
                 orderBy('phi_Province', 'phi_Confirmed', 'phi_Date')

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [5]:
from pyspark.sql.functions import date_format, to_date

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [6]:
phi_df2 = phi_df1.withColumn("phi_Date", date_format(to_date(phi_df1.phi_Date, "dd-MM-yyyy"), "yyyy-MM-dd"))
phi_df2.show()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+----------+------------+-------------+----------+
|  phi_Date|phi_Province|phi_Confirmed|phi_Deaths|
+----------+------------+-------------+----------+
|2020-03-08|     Alberta|            1|         0|
|2020-03-09|     Alberta|            7|         0|
|2020-03-11|     Alberta|           14|         0|
|2020-03-12|     Alberta|           19|         0|
|2020-03-13|     Alberta|           23|         0|
|2020-03-14|     Alberta|           29|         0|
|2020-03-15|     Alberta|           39|         0|
|2020-03-16|     Alberta|           56|         0|
|2020-03-17|     Alberta|           74|         0|
|2020-03-18|     Alberta|           97|         0|
|2020-03-19|     Alberta|          146|         1|
|2020-03-20|     Alberta|          195|         1|
|2020-03-21|     Alberta|          226|         1|
|2020-03-22|     Alberta|          259|         1|
|2020-03-23|     Alberta|          301|         1|
|2020-03-24|     Alberta|          358|         2|
|2020-03-25|     Alberta|      

In [7]:
phi_df2.printSchema()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
 |-- phi_Date: string (nullable = true)
 |-- phi_Province: string (nullable = true)
 |-- phi_Confirmed: integer (nullable = true)
 |-- phi_Deaths: integer (nullable = true)

In [8]:
jh_df1 = jh_df.filter(jh_df.jh_Country == 'Canada')

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [9]:
phi_df3 = phi_df2.where("phi_Province NOT IN ('Nunavut', 'Repatriated travellers', 'Canada')")

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [10]:
jh_df2 = jh_df1.where("jh_Province NOT IN ('Grand Princess', 'Diamond Princess')")

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [20]:
jh_df2.show()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+----------+----------+-----------+-------+---------+------------+------------+---------+
|   jh_Date|jh_Country|jh_Province| jh_Lat|  jh_Long|jh_Confirmed|jh_Recovered|jh_Deaths|
+----------+----------+-----------+-------+---------+------------+------------+---------+
|2020-01-22|    Canada|    Alberta|53.9333|-116.5765|           0|        null|        0|
|2020-01-23|    Canada|    Alberta|53.9333|-116.5765|           0|        null|        0|
|2020-01-24|    Canada|    Alberta|53.9333|-116.5765|           0|        null|        0|
|2020-01-25|    Canada|    Alberta|53.9333|-116.5765|           0|        null|        0|
|2020-01-26|    Canada|    Alberta|53.9333|-116.5765|           0|        null|        0|
|2020-01-27|    Canada|    Alberta|53.9333|-116.5765|           0|        null|        0|
|2020-01-28|    Canada|    Alberta|53.9333|-116.5765|           0|        null|        0|
|2020-01-29|    Canada|    Alberta|53.9333|-116.5765|           0|        null|        0|
|2020-01-3

In [11]:
jh_df2.printSchema()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
 |-- jh_Date: date (nullable = true)
 |-- jh_Country: string (nullable = true)
 |-- jh_Province: string (nullable = true)
 |-- jh_Lat: double (nullable = true)
 |-- jh_Long: double (nullable = true)
 |-- jh_Confirmed: integer (nullable = true)
 |-- jh_Recovered: integer (nullable = true)
 |-- jh_Deaths: integer (nullable = true)

In [12]:
phi_df3.printSchema()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
 |-- phi_Date: string (nullable = true)
 |-- phi_Province: string (nullable = true)
 |-- phi_Confirmed: integer (nullable = true)
 |-- phi_Deaths: integer (nullable = true)

In [13]:
dfLeftOuterJoin = phi_df3. \
    join(jh_df2, \
         (phi_df3.phi_Date == jh_df2.jh_Date) & (phi_df3.phi_Province == jh_df2.jh_Province), \
         'left' \
        )

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [38]:
df_Final = dfLeftOuterJoin.select(dfLeftOuterJoin.phi_Date.alias('Date'), \
                                  dfLeftOuterJoin.jh_Country.alias('Country'), \
                                  dfLeftOuterJoin.phi_Province.alias('Province'), \
                                  dfLeftOuterJoin.jh_Lat.alias('Latitude'), \
                                  dfLeftOuterJoin.jh_Long.alias('Longitude'), \
                                  dfLeftOuterJoin.phi_Confirmed.alias('Confirmed'), \
                                  dfLeftOuterJoin.jh_Recovered.alias('Recovered'), \
                                  dfLeftOuterJoin.jh_Deaths.alias('Deaths') \
                                 ). \
                            orderBy('Province', 'Confirmed', 'Date')

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [39]:
df_Final.show()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+----------+-------+--------+--------+---------+---------+---------+------+
|      Date|Country|Province|Latitude|Longitude|Confirmed|Recovered|Deaths|
+----------+-------+--------+--------+---------+---------+---------+------+
|2020-03-08| Canada| Alberta| 53.9333|-116.5765|        1|     null|     0|
|2020-03-09| Canada| Alberta| 53.9333|-116.5765|        7|     null|     0|
|2020-03-11| Canada| Alberta| 53.9333|-116.5765|       14|     null|     0|
|2020-03-12| Canada| Alberta| 53.9333|-116.5765|       19|     null|     0|
|2020-03-13| Canada| Alberta| 53.9333|-116.5765|       23|     null|     0|
|2020-03-14| Canada| Alberta| 53.9333|-116.5765|       29|     null|     0|
|2020-03-15| Canada| Alberta| 53.9333|-116.5765|       39|     null|     0|
|2020-03-16| Canada| Alberta| 53.9333|-116.5765|       56|     null|     0|
|2020-03-17| Canada| Alberta| 53.9333|-116.5765|       74|     null|     0|
|2020-03-18| Canada| Alberta| 53.9333|-116.5765|       97|     null|     0|
|2020-03-19|

In [40]:
df_Final.registerTempTable('hive_temp')

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [41]:
spark.sql('select* from hive_temp').show()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+----------+-------+--------+--------+---------+---------+---------+------+
|      Date|Country|Province|Latitude|Longitude|Confirmed|Recovered|Deaths|
+----------+-------+--------+--------+---------+---------+---------+------+
|2020-03-08| Canada| Alberta| 53.9333|-116.5765|        1|     null|     0|
|2020-03-09| Canada| Alberta| 53.9333|-116.5765|        7|     null|     0|
|2020-03-11| Canada| Alberta| 53.9333|-116.5765|       14|     null|     0|
|2020-03-12| Canada| Alberta| 53.9333|-116.5765|       19|     null|     0|
|2020-03-13| Canada| Alberta| 53.9333|-116.5765|       23|     null|     0|
|2020-03-14| Canada| Alberta| 53.9333|-116.5765|       29|     null|     0|
|2020-03-15| Canada| Alberta| 53.9333|-116.5765|       39|     null|     0|
|2020-03-16| Canada| Alberta| 53.9333|-116.5765|       56|     null|     0|
|2020-03-17| Canada| Alberta| 53.9333|-116.5765|       74|     null|     0|
|2020-03-18| Canada| Alberta| 53.9333|-116.5765|       97|     null|     0|
|2020-03-19|

In [70]:
spark.sql('DROP TABLE IF EXISTS hive_table')
spark.sql('CREATE TABLE hive_table (FinalDate string, Country string, Province string, Latitude double, Longitude double, \
           Confirmed int, Recovered int, Deaths int) \
           USING CSV OPTIONS (path "s3://covid-19-tracker-2020/hive/tables/hive-table.csv", header "true")')

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

DataFrame[]

In [71]:
spark.sql('DESCRIBE hive_table').show()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+---------+---------+-------+
| col_name|data_type|comment|
+---------+---------+-------+
|FinalDate|   string|   null|
|  Country|   string|   null|
| Province|   string|   null|
| Latitude|   double|   null|
|Longitude|   double|   null|
|Confirmed|      int|   null|
|Recovered|      int|   null|
|   Deaths|      int|   null|
+---------+---------+-------+

In [74]:
spark.sql('INSERT INTO hive_table SELECT* FROM hive_temp')

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

DataFrame[]

In [73]:
spark.sql('SELECT* FROM hive_table').show()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+----------+-------+----------------+--------+---------+---------+---------+------+
| FinalDate|Country|        Province|Latitude|Longitude|Confirmed|Recovered|Deaths|
+----------+-------+----------------+--------+---------+---------+---------+------+
|2020-01-31| Canada|         Ontario| 51.2538| -85.3232|        3|     null|     0|
|2020-01-31| Canada|British Columbia| 53.7267|-127.6476|        1|     null|     0|
|2020-02-08| Canada|         Ontario| 51.2538| -85.3232|        3|     null|     0|
|2020-02-08| Canada|British Columbia| 53.7267|-127.6476|        4|     null|     0|
|2020-02-16| Canada|         Ontario| 51.2538| -85.3232|        3|     null|     0|
|2020-02-16| Canada|British Columbia| 53.7267|-127.6476|        5|     null|     0|
|2020-02-21| Canada|         Ontario| 51.2538| -85.3232|        3|     null|     0|
|2020-02-21| Canada|British Columbia| 53.7267|-127.6476|        6|     null|     0|
|2020-02-24| Canada|         Ontario| 51.2538| -85.3232|        4|     null|